# Step 1: Import Required Libraries

In [8]:
import os
import pandas as pd
import numpy as np
# For API calls
import requests
# For database storage
import sqlite3
import matplotlib as plt

# Step 2: Load Data

In [ ]:
## Load movie metadata from TMDb API
def fetch_tmdb_data(movie_title):
    """
    Fetch metadata for a given movie title from TMDb API.
    """
    api_key = "your_tmdb_api_key"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_title}"
    response = requests.get(url).json()
    return response

## Load supplementary movie data from OMDb API
def fetch_omdb_data(movie_title):
    """
    Fetch additional movie details from OMDb API.
    """
    api_key = "your_omdb_api_key" # From api.py file
    url = f"http://www.omdbapi.com/?apikey={api_key}&t={movie_title}"
    response = requests.get(url).json()
    return response

## Load Academy Award winners dataset from CSV
awards_df = pd.read_csv("academy_awards.csv")

## Load acceptance speech transcripts from text files or a CSV dataset
speech_df = pd.read_csv("oscar_speeches.csv")



# Step 3: Data Cleaning

In [ ]:
## Clean movie metadata
def clean_movie_data(df):
    """
    Handle missing values and standardize column names in the movie dataset.
    """
    df.dropna(subset=["title", "release_year"], inplace=True)
    df.fillna({"box_office": 0, "runtime": df["runtime"].median()}, inplace=True)
    return df

## Clean speech transcripts
# def preprocess_speech_text(text):
#     """
#     Tokenize and clean Oscar acceptance speech text for word frequency analysis.
#     """
#     nltk.download("stopwords")
#     nltk.download("punkt")
#     tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
#     filtered_words = [word for word in tokens if word.isalnum() and word not in stopwords.words("english")]
#     return " ".join(filtered_words)

# speech_df["cleaned_speech"] = speech_df["speech_text"].apply(preprocess_speech_text)



In [ ]:
# Step 4: Store Data in SQLite Database
conn = sqlite3.connect("academy_awards.db")
awards_df.to_sql("awards", conn, if_exists="replace", index=False)
speech_df.to_sql("speeches", conn, if_exists="replace", index=False)



In [ ]:
# Step 5: SQL Queries & Analysis
## Query genres of Best Picture winners over decades
query = """
SELECT genre, COUNT(*) AS num_wins, strftime('%Y', award_year) AS decade
FROM awards
WHERE category = 'Best Picture'
GROUP BY genre, decade
ORDER BY decade ASC;
"""
genre_trends_df = pd.read_sql(query, conn)

## Query word frequency in acceptance speeches
query = """
SELECT cleaned_speech FROM speeches;
"""
speech_texts = pd.read_sql(query, conn)



In [ ]:
# Step 6: Data Visualization
## Bar Chart - Best Picture Wins by Genre
plt.figure(figsize=(12,6))
sns.barplot(x="genre", y="num_wins", hue="decade", data=genre_trends_df)
plt.xticks(rotation=45)
plt.title("Best Picture Wins by Genre Over Decades")
plt.show()

## Scatter Plot - Box Office vs IMDb Ratings
plt.figure(figsize=(10,5))
sns.scatterplot(x="box_office", y="imdb_rating", hue="decade", data=awards_df)
plt.title("Box Office Revenue vs IMDb Ratings for Oscar Winners")
plt.show()

## Word Cloud - Common Words in Acceptance Speeches
all_text = " ".join(speech_texts["cleaned_speech"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_text)
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Most Common Words in Oscar Acceptance Speeches")
plt.show()



In [ ]:
# Step 7: Conclusion & Interpretation
"""
- The bar chart shows which genres have dominated the Best Picture category over time.
- The scatter plot identifies any correlation between box office revenue and audience reception (IMDb ratings).
- The word cloud highlights common themes in Oscar speeches, reflecting industry trends and sentiments.
"""



In [ ]:
# Step 8: Save & Commit Changes to GitHub
"""
1. Regularly push updates to GitHub with meaningful commit messages.
2. Include a README file explaining data sources, analysis, and visualizations.
3. Ensure documentation is clear and reproducible for others.
"""



### Overview of the Analysis (examples)
- In this analysis, we explored the relationship between the race of law enforcement officers and the race of the drivers they stop. Our goal was to see if there’s any indication of bias in traffic stops based on the racial identity of the officers. To do this, we used a chi-squared test for independence, which helps us understand whether there’s a meaningful connection between these two groups.

### Results of the Chi-Squared Test
- **Chi-Squared Statistic:** We calculated a chi-squared statistic of 122.92. This high number shows that there’s a significant difference between the actual number of stops for different racial groups and what we would expect to see if there were no connection between the officer's race and the driver's race. In other words, this suggests that the patterns we observe in the data are unlikely to be just a coincidence.

- **P-Value:** The p-value we found was about 8.20e-17, which is extremely low. This tells us that the result is statistically significant since it’s much lower than the usual thresholds (like 0.05 or 0.01). A low p-value means we have strong evidence against the idea that there’s no connection between the officer's race and the driver's race.

### Interpretation of Findings
- The results show a strong connection between the race of the officer and the race of the driver being stopped. This means that a driver's chances of being stopped may change depending on the officer's race, suggesting there might be some bias in how traffic stops are carried out.

### Implications
- These findings are important for understanding how race plays a role in law enforcement. They suggest that different racial groups might be treated differently by officers during traffic stops. It's crucial to address these biases to ensure fairness and equality in policing.

### Conclusion
- The strong evidence from the chi-squared statistic and p-value emphasizes the importance of further examining law enforcement practices. Police leaders and community advocacy groups should take these findings into account when reviewing policies and training programs designed to reduce racial bias in policing.